In [382]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics 
from sklearn import svm

In [408]:
#read in the data using pandas

df = pd.read_csv(Path.cwd().parent / "data" / "tracks_with_explict_content_from_api_and_crawler.csv")
to_drop = [col for col in df.columns if "Unnamed" in col.split(":")] + ["track_uri", "artist_genres", "track_name", "album"]


# df = pd.read_csv(Path.cwd().parent / "data" / "final_dataset.csv")
#check data has been read in properly
# to_drop = [col for col in df.columns if "Unnamed" in col.split(":")] + ["track_uri", "artist_genres"]
# to_drop += [col for i, col in enumerate(df.columns) if i > 14]

to_drop

['Unnamed: 0', 'track_uri', 'artist_genres', 'track_name', 'album']

In [409]:
df = df.drop(columns=to_drop)

In [405]:
X = df.drop(columns=["is_explict_content"])
y = df["is_explict_content"]

In [410]:
df.dtypes

artist_name            object
artist_popularity       int64
artist_followers        int64
track_popularity        int64
track_danceability    float64
track_energy          float64
track_loudness        float64
track_tempo           float64
track_duration_ms       int64
is_explict_content    float64
dtype: object

In [411]:
# TEMP!

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["artist_name"] = le.fit_transform(df["artist_name"])

# Remove Outliers

In [412]:
df.shape

(8250, 10)

In [413]:
df.dropna(inplace=True)

In [414]:
df.shape

(6759, 10)

In [386]:
df["country_code"] = df["country_code"].astype('category')
# df["artist"] = df["artist"].astype('category')

In [415]:
# take all numerics
quatiative = df.select_dtypes(exclude=['category'])

# boundaries for chaking outliers
upper_bound = quatiative.mean() + 3 * quatiative.std()
lower_bound = quatiative.mean() - 3 * quatiative.std()

outliers = []

for num, row in quatiative.iterrows():
    outliers_in_row = [1 for col in quatiative.columns if row[col] > upper_bound[col] or row[col] < lower_bound[col]].count(1)    
    # add this index(row) if 30% of it's content is outlaiered
    if outliers_in_row > 0.1 * len(row):
        outliers.append(num)

print(f"Number of indexes with 30% outliers of it's content are: {len(outliers)}")

Number of indexes with 30% outliers of it's content are: 21


In [416]:
df.drop(outliers, inplace=True)

df.shape

(6738, 10)

# Normalization

In [389]:
df.describe()

,artist_popularity,artist_followers,track_popularity,track_danceability,track_energy,track_loudness,track_tempo,track_duration_ms,is_explict_content,artist,...,alt z,hip pop,motown,quiet storm,funk,singer-songwriter,folk,disco,neo mellow,trap
count,4908.000000,4.908000e+03,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,...,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.00000,4908.000000
mean,74.078240,8.009317e+06,56.365933,0.605179,0.660392,-6.964303,120.483805,221207.459454,0.184189,869.614914,...,0.029747,0.023024,0.006112,0.002649,0.008965,0.016504,0.015892,0.003871,0.01467,0.042380
std,12.702711,1.485474e+07,26.129261,0.159833,0.204606,3.351880,29.167023,60666.539805,0.387678,518.669114,...,0.169907,0.149994,0.077951,0.051403,0.094268,0.127415,0.125072,0.062105,0.12024,0.201475
min,17.000000,2.210000e+02,0.000000,0.089100,0.016200,-30.447000,47.859000,15160.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,66.000000,4.958540e+05,49.000000,0.498000,0.527000,-8.261000,96.951500,183408.250000,0.000000,444.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,75.000000,2.082301e+06,64.000000,0.612000,0.690000,-6.233000,119.957000,211667.000000,0.000000,851.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,83.000000,7.083105e+06,74.000000,0.723000,0.823000,-4.713750,139.795500,247033.250000,0.000000,1323.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,100.000000,9.006724e+07,98.000000,0.981000,0.996000,-0.140000,210.108000,890000.000000,1.000000,1814.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [390]:
# from sklearn.preprocessing import MinMaxScaler
# cols_to_scale = ["artist_popularity","artist_followers","track_popularity", "track_tempo", "track_loudness"] #, "track_duration_ms"]
# for c in cols_to_scale:
#     scaler = MinMaxScaler() 
#     scaled_values = scaler.fit_transform(df[c].to_numpy().reshape(-1, 1)) 
#     df.loc[:,c] = scaled_values

In [391]:
df.describe()

,artist_popularity,artist_followers,track_popularity,track_danceability,track_energy,track_loudness,track_tempo,track_duration_ms,is_explict_content,artist,...,alt z,hip pop,motown,quiet storm,funk,singer-songwriter,folk,disco,neo mellow,trap
count,4908.000000,4.908000e+03,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,...,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.000000,4908.00000,4908.000000
mean,74.078240,8.009317e+06,56.365933,0.605179,0.660392,-6.964303,120.483805,221207.459454,0.184189,869.614914,...,0.029747,0.023024,0.006112,0.002649,0.008965,0.016504,0.015892,0.003871,0.01467,0.042380
std,12.702711,1.485474e+07,26.129261,0.159833,0.204606,3.351880,29.167023,60666.539805,0.387678,518.669114,...,0.169907,0.149994,0.077951,0.051403,0.094268,0.127415,0.125072,0.062105,0.12024,0.201475
min,17.000000,2.210000e+02,0.000000,0.089100,0.016200,-30.447000,47.859000,15160.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,66.000000,4.958540e+05,49.000000,0.498000,0.527000,-8.261000,96.951500,183408.250000,0.000000,444.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,75.000000,2.082301e+06,64.000000,0.612000,0.690000,-6.233000,119.957000,211667.000000,0.000000,851.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,83.000000,7.083105e+06,74.000000,0.723000,0.823000,-4.713750,139.795500,247033.250000,0.000000,1323.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,100.000000,9.006724e+07,98.000000,0.981000,0.996000,-0.140000,210.108000,890000.000000,1.000000,1814.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [417]:
X = df.drop(columns=["is_explict_content"])
y = df["is_explict_content"]

# All in one

In [418]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [421]:
clf1 = DecisionTreeClassifier()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = KNeighborsClassifier(n_neighbors=13)
alg_names = ["decision tree", "random forest", "KNN"]

In [423]:
for idx, clf in enumerate([clf1, clf2, clf3]):
    clf.fit(X_train, y_train)
    y_pred_train = clf.predict(X_train)
    y_pred = clf.predict(X_test)
    
    print(clf.feature_importances_)
    
#     print(alg_names[idx])
#     print("==========================")
#     print(f"\tTrain data")
#     print("\t==========================")
#     print(f"{metrics.classification_report(y_true = y_train, y_pred = y_pred_train)}")
#     print(f"\tTest data")
#     print("\t==========================")
#     print(f"{metrics.classification_report(y_true = y_test, y_pred = y_pred)}")


#     print('f1 on Train data= ', metrics.f1_score(y_true = y_train, y_pred = y_pred_train))
#     print('f1 on test data= ', metrics.f1_score(y_true = y_test, y_pred = y_pred))
#     print('Accuracy on Train data= ', metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train))
#     print('Accuracy on test data= ', metrics.accuracy_score(y_true = y_test, y_pred = y_pred))
    print()

print(df.shape)

[0.07314626 0.09652488 0.13849484 0.07046407 0.16821892 0.10350275
 0.1255766  0.12282401 0.10124768]

[0.10086872 0.08929855 0.11893065 0.07565492 0.16772155 0.10372489
 0.10479711 0.13243154 0.10657208]



AttributeError: 'KNeighborsClassifier' object has no attribute 'feature_importances_'

In [396]:
from sklearn.model_selection import GridSearchCV

params_decision_tree = {"max_depth": [2,4,6,8,10], "min_samples_split": [3,5,7,10,15,20]}
params_random_forest = {"n_estimators": [50, 100,200,300]}
params_knn = {"n_neighbors": range(3, 20)}
params_nb = {}

clf_params = [(DecisionTreeClassifier(), params_decision_tree),
             (RandomForestClassifier(), params_random_forest), 
             (KNeighborsClassifier(), params_knn),
             (GaussianNB(), params_nb)]

for idx, (clf, params) in enumerate(clf_params):
    clf_cv = GridSearchCV(clf,params,cv=10)
    clf_cv.fit(X,y)
    print(alg_names[idx])
    print("==========================")
    print(f"best params are: {clf_cv.best_params_}")
    print(f"best score is: {clf_cv.best_score_}")

decision tree
best params are: {'max_depth': 10, 'min_samples_split': 3}
best score is: 0.8614431189991271
random forest
best params are: {'n_estimators': 300}
best score is: 0.8924140654225029
KNN
best params are: {'n_neighbors': 10}
best score is: 0.8312993058730621


IndexError: list index out of range